In [ ]:
# Image Generator Testing Notebook

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from logger import LOG

# Assume OpenAI API key is already set in environment
API_KEY = os.getenv("OPENAI_API_KEY")

if not API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable")

# Load prompt
PROMPT_FILE = "./prompts/image_generator_prompt.txt"

def load_prompt(prompt_file=PROMPT_FILE):
    try:
        with open(prompt_file, "r", encoding="utf-8") as file:
            return file.read().strip()
    except FileNotFoundError:
        raise FileNotFoundError(f"Prompt file '{prompt_file}' not found!")

# Initialize prompt and chatbot
prompt = load_prompt()

# Create ChatBot with LangChain
system_prompt = ChatPromptTemplate.from_messages([
    ("system", prompt),  # System prompt
    ("user", "{input}")  # User input placeholder
])

# Initialize ChatOpenAI model
chat_model = ChatOpenAI(
    model="gpt-4o",
    temperature=0.5,
    max_tokens=300
)

# Combine prompt and model to create the chatbot
chatbot = system_prompt | chat_model

# Wrap the chatbot with message history handling
chatbot_with_history = RunnableWithMessageHistory(chatbot)

# Test prompt
user_input = "A beautiful landscape with mountains and a river."

# Generate image description using the chatbot
response = chatbot_with_history.invoke([HumanMessage(content=user_input)])

# Display the response if successful
if response:
    LOG.debug(f"Generated Description: {response.content}")
    print(response.content)
else:
    print("Image generation failed.")
